# EGS master catalogue

This notebook presents the merge of the various pristine catalogues to produce HELP mater catalogue on EGS.

In [ ]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))
import datetime
print("This notebook was executed on: \n{}".format(datetime.datetime.now()))

In [ ]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

import os
import time

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np
from pymoc import MOC

from herschelhelp_internal.masterlist import merge_catalogues, nb_merge_dist_plot, specz_merge
from herschelhelp_internal.utils import coords_to_hpidx, ebv, gen_help_id, inMoc

In [ ]:
TMP_DIR = os.environ.get('TMP_DIR', "./data_tmp")
OUT_DIR = os.environ.get('OUT_DIR', "./data")
SUFFIX = os.environ.get('SUFFIX', time.strftime("_%Y%m%d"))

try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

## I - Reading the prepared pristine catalogues

In [ ]:
aegis = Table.read("{}/AEGIS.fits".format(TMP_DIR))
candels = Table.read("{}/CANDELS-EGS.fits".format(TMP_DIR))
wirds = Table.read("{}/CFHT-WIRDS.fits".format(TMP_DIR))
cfhtls_wide = Table.read("{}/CFHTLS-WIDE.fits".format(TMP_DIR))
cfhtls_deep = Table.read("{}/CFHTLS-DEEP.fits".format(TMP_DIR))
cfhtlens = Table.read("{}/CFHTLENS.fits".format(TMP_DIR))
deep = Table.read("{}/DEEP2.fits".format(TMP_DIR))
irac = Table.read("{}/IRAC-EGS.fits".format(TMP_DIR))
hsc = Table.read("{}/HSC.fits".format(TMP_DIR))
ps1 = Table.read("{}/PS1.fits".format(TMP_DIR))
legacy = Table.read("{}/LegacySurvey.fits".format(TMP_DIR))
uhs = Table.read("{}/UHS.fits".format(TMP_DIR))

## II - Merging tables

We first merge the optical catalogues and then add the infrared ones.

At every step, we look at the distribution of the distances to the nearest source in the merged catalogue to determine the best crossmatching radius.

### HSC

In [ ]:
master_catalogue = hsc
master_catalogue['hsc_ra'].name = 'ra'
master_catalogue['hsc_dec'].name = 'dec'

### Add PanSTARRS

In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(ps1['ps1_ra'], ps1['ps1_dec'])
)

In [ ]:
# Given the graph above, we use 0.8 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, ps1, "ps1_ra", "ps1_dec", radius=0.8*u.arcsec)

### AEGIS

In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(aegis['aegis_ra'], aegis['aegis_dec'])
)

In [ ]:
# Given the graph above, we use 0.8 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, aegis, "aegis_ra", "aegis_dec", radius=0.8*u.arcsec)

### CANDELS

In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(candels['candels-egs_ra'], candels['candels-egs_dec'])
)

In [ ]:
# Given the graph above, we use 0.8 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, candels, "candels-egs_ra", "candels-egs_dec", radius=0.8*u.arcsec)

### CFHT-WIRDS

In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(wirds['wirds_ra'], wirds['wirds_dec'])
)

In [ ]:
# Given the graph above, we use 0.8 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, wirds, "wirds_ra", "wirds_dec", radius=0.8*u.arcsec)

### CFHTLS-WIDE

In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(cfhtls_wide['cfhtls-wide_ra'], cfhtls_wide['cfhtls-wide_dec'])
)

In [ ]:
# Given the graph above, we use 0.8 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, 
                                    cfhtls_wide, 
                                    "cfhtls-wide_ra", 
                                    "cfhtls-wide_dec", 
                                    radius=0.8*u.arcsec)

### CFHTLS-DEEP

In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(cfhtls_deep['cfhtls-deep_ra'], cfhtls_deep['cfhtls-deep_dec'])
)

In [ ]:
# Given the graph above, we use 0.8 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, 
                                    cfhtls_deep, 
                                    "cfhtls-deep_ra", 
                                    "cfhtls-deep_dec", 
                                    radius=0.8*u.arcsec)

### CFHTLenS

In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(cfhtlens['cfhtlens_ra'], cfhtlens['cfhtlens_dec'])
)

In [ ]:
# Given the graph above, we use 0.8 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, cfhtlens, "cfhtlens_ra", "cfhtlens_dec", radius=0.8*u.arcsec)

### DEEP2

In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(deep['deep2_ra'], deep['deep2_dec'])
)

In [ ]:
# Given the graph above, we use 0.8 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, deep, "deep2_ra", "deep2_dec", radius=0.8*u.arcsec)

### IRAC-EGS

In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(irac['irac-egs_ra'], irac['irac-egs_dec'])
)

In [ ]:
# Given the graph above, we use 0.8 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, irac, "irac-egs_ra", "irac-egs_dec", radius=0.8*u.arcsec)

### Legacy Survey

In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(legacy['legacy_ra'], legacy['legacy_dec'])
)

In [ ]:
# Given the graph above, we use 0.8 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, legacy, "legacy_ra", "legacy_dec", radius=0.8*u.arcsec)

### UHS

In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(uhs['uhs_ra'], uhs['uhs_dec'])
)

In [ ]:
# Given the graph above, we use 0.8 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, uhs, "uhs_ra", "uhs_dec", radius=0.8*u.arcsec)

### Cleaning

When we merge the catalogues, astropy masks the non-existent values (e.g. when a row comes only from a catalogue and has no counterparts in the other, the columns from the latest are masked for that row). We indicate to use NaN for masked values for floats columns, False for flag columns and -1 for ID columns.

In [ ]:
for col in master_catalogue.colnames:
    if "m_" in col or "merr_" in col or "f_" in col or "ferr_" in col or "stellarity" in col:
        master_catalogue[col].fill_value = np.nan
    elif "flag" in col:
        master_catalogue[col].fill_value = 0
    elif "id" in col:
        master_catalogue[col].fill_value = -1
        
master_catalogue = master_catalogue.filled()

In [ ]:
master_catalogue[:10].show_in_notebook()

## III - Merging flags and stellarity

Each pristine catalogue contains a flag indicating if the source was associated to a another nearby source that was removed during the cleaning process.  We merge these flags in a single one.

In [ ]:
flag_cleaned_columns = [column for column in master_catalogue.colnames
                        if 'flag_cleaned' in column]

flag_column = np.zeros(len(master_catalogue), dtype=bool)
for column in flag_cleaned_columns:
    flag_column |= master_catalogue[column]
    
master_catalogue.add_column(Column(data=flag_column, name="flag_cleaned"))
master_catalogue.remove_columns(flag_cleaned_columns)

Each pristine catalogue contains a flag indicating the probability of a source being a Gaia object (0: not a Gaia object, 1: possibly, 2: probably, 3: definitely).  We merge these flags taking the highest value.

In [ ]:
flag_gaia_columns = [column for column in master_catalogue.colnames
                     if 'flag_gaia' in column]

master_catalogue.add_column(Column(
    data=np.max([master_catalogue[column] for column in flag_gaia_columns], axis=0),
    name="flag_gaia"
))
master_catalogue.remove_columns(flag_gaia_columns)

Each prisitine catalogue may contain one or several stellarity columns indicating the probability (0 to 1) of each source being a star.  We merge these columns taking the highest value.  We keep trace of the origin of the stellarity.

In [ ]:
stellarity_columns = [column for column in master_catalogue.colnames
                      if 'stellarity' in column]

print(", ".join(stellarity_columns))

In [ ]:
# We create an masked array with all the stellarities and get the maximum value, as well as its
# origin.  Some sources may not have an associated stellarity.
stellarity_array = np.array([master_catalogue[column] for column in stellarity_columns])
stellarity_array = np.ma.masked_array(stellarity_array, np.isnan(stellarity_array))

max_stellarity = np.max(stellarity_array, axis=0)
max_stellarity.fill_value = np.nan

no_stellarity_mask = max_stellarity.mask

master_catalogue.add_column(Column(data=max_stellarity.filled(), name="stellarity"))

stellarity_origin = np.full(len(master_catalogue), "NO_INFORMATION", dtype="S20")
stellarity_origin[~no_stellarity_mask] = np.array(stellarity_columns)[np.argmax(stellarity_array, axis=0)[~no_stellarity_mask]]

master_catalogue.add_column(Column(data=stellarity_origin, name="stellarity_origin"))

master_catalogue.remove_columns(stellarity_columns)

## IV - Adding E(B-V) column

In [ ]:
master_catalogue.add_column(
    ebv(master_catalogue['ra'], master_catalogue['dec'])
)

## V - Adding HELP unique identifiers and field columns

In [ ]:
master_catalogue.add_column(Column(gen_help_id(master_catalogue['ra'], master_catalogue['dec']),
                                   name="help_id"))
master_catalogue.add_column(Column(np.full(len(master_catalogue), "EGS", dtype='<U18'),
                                   name="field"))

In [ ]:
# Check that the HELP Ids are unique
if len(master_catalogue) != len(np.unique(master_catalogue['help_id'])):
    print("The HELP IDs are not unique!!!")
else:
    print("OK!")

## VI - Cross-matching with spec-z catalogue

In [ ]:
specz =  Table.read("../../dmu23/dmu23_EGS/data/EGS-specz-v2.1.csv")
for col in specz.colnames:
    specz[col].name = specz[col].name.lower()

specz['objid'].name = 'specz_id'


In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(specz['ra'] * u.deg, specz['dec'] * u.deg)
)

In [ ]:
master_catalogue = specz_merge(master_catalogue, specz, radius=1. * u.arcsec)

## VII - Choosing between multiple values for the same filter

There are many different bands to choose between here.

### CFHT Megacam

CFHT-WIRDS is the only survey that has J, H and Ks so we take them directly. After that we need to select ugriz bands from between CFHTLS, CFHT-WIRDS and CFHTLenS. We take these in order of depth.

|  Survey     | Bands         | Notes|
|:------------|:--------------|:-----|
| CFHTLS-DEEP | u, g, r, i, z |      |
| CFHTLS-WIDE | u, g, r, i, z |      |
| CFHT-WIRDS  | u, g, r, i, z | Ks selected so may have unique objects       |
| CFHTLenS    | u, g, r, i, z | Reprocessing of CFHTLS-WIDE so not used      |
| CANDELS-EGS | u, g, r, i, z | Priors from very deep data so may have unique objects     |
| IRAC-EGS | u, g, r, i, z | Priors from IRAC so may have unique objects     |

In [ ]:
megacam_origin = Table()
megacam_origin.add_column(master_catalogue['help_id'])

In [ ]:
megacam_stats = Table()
megacam_stats.add_column(Column(data=['u','g','r','i','z'], name="Band"))
for col in ["CFHTLS-DEEP", "CFHTLS-WIDE", "CFHT-WIRDS", "CFHTLenS", "CANDELS", "IRAC-EGS"]:
    megacam_stats.add_column(Column(data=np.full(6, 0), name="{}".format(col)))
    megacam_stats.add_column(Column(data=np.full(6, 0), name="use {}".format(col)))
    megacam_stats.add_column(Column(data=np.full(6, 0), name="{} ap".format(col)))
    megacam_stats.add_column(Column(data=np.full(6, 0), name="use {} ap".format(col)))

In [ ]:
megacam_bands = ['u','g','r','i','z'] 
for band in megacam_bands:

    # Megacam total flux 
    has_cfhtls_deep = ~np.isnan(master_catalogue['f_cfhtls-deep_' + band])
    has_cfhtls_wide = ~np.isnan(master_catalogue['f_cfhtls-wide_' + band])
    has_wirds = ~np.isnan(master_catalogue['f_wirds_' + band])
    has_cfhtlens = ~np.isnan(master_catalogue['f_cfhtlens_' + band])
    has_candels = ~np.isnan(master_catalogue['f_candels-megacam_' + band])
    has_irac = ~np.isnan(master_catalogue['f_irac-megacam_' + band])
    

    use_cfhtls_deep = has_cfhtls_deep 
    use_cfhtls_wide = has_cfhtls_wide & ~has_cfhtls_deep 
    use_wirds =       has_wirds       & ~has_cfhtls_deep & ~has_cfhtls_wide  
    use_cfhtlens =    np.zeros(len(master_catalogue), dtype=bool) #We still merge CFHTLenS to keep ids in cross id
    use_candels =     has_candels     & ~has_cfhtls_deep & ~has_cfhtls_wide & ~has_wirds & ~has_cfhtlens 
    use_irac = has_irac &~has_candels     & ~has_cfhtls_deep & ~has_cfhtls_wide & ~has_wirds & ~has_cfhtlens 

    f_megacam = np.full(len(master_catalogue), np.nan)
    f_megacam[use_cfhtls_deep] = master_catalogue['f_cfhtls-deep_' + band][use_cfhtls_deep]
    f_megacam[use_cfhtls_wide] = master_catalogue['f_cfhtls-wide_' + band][use_cfhtls_wide]
    f_megacam[use_wirds] = master_catalogue['f_wirds_' + band][use_wirds]
    f_megacam[use_cfhtlens] = master_catalogue['f_cfhtlens_' + band][use_cfhtlens]
    f_megacam[use_candels] = master_catalogue['f_candels-megacam_' + band][use_candels]
    f_megacam[use_irac] = master_catalogue['f_irac-megacam_' + band][use_irac]

    ferr_megacam = np.full(len(master_catalogue), np.nan)
    ferr_megacam[use_cfhtls_deep] = master_catalogue['ferr_cfhtls-deep_' + band][use_cfhtls_deep]
    ferr_megacam[use_cfhtls_wide] = master_catalogue['ferr_cfhtls-wide_' + band][use_cfhtls_wide]
    ferr_megacam[use_wirds] = master_catalogue['ferr_wirds_' + band][use_wirds]
    ferr_megacam[use_cfhtlens] = master_catalogue['ferr_cfhtlens_' + band][use_cfhtlens]
    ferr_megacam[use_candels] = master_catalogue['ferr_candels-megacam_' + band][use_candels]
    ferr_megacam[use_irac] = master_catalogue['ferr_irac-megacam_' + band][use_irac]
    
    m_megacam = np.full(len(master_catalogue), np.nan)
    m_megacam[use_cfhtls_deep] = master_catalogue['m_cfhtls-deep_' + band][use_cfhtls_deep]
    m_megacam[use_cfhtls_wide] = master_catalogue['m_cfhtls-wide_' + band][use_cfhtls_wide]
    m_megacam[use_wirds] = master_catalogue['m_wirds_' + band][use_wirds]
    m_megacam[use_cfhtlens] = master_catalogue['m_cfhtlens_' + band][use_cfhtlens]
    m_megacam[use_candels] = master_catalogue['m_candels-megacam_' + band][use_candels]
    m_megacam[use_irac] = master_catalogue['m_irac-megacam_' + band][use_irac]

    merr_megacam = np.full(len(master_catalogue), np.nan)
    merr_megacam[use_cfhtls_deep] = master_catalogue['merr_cfhtls-deep_' + band][use_cfhtls_deep]
    merr_megacam[use_cfhtls_wide] = master_catalogue['merr_cfhtls-wide_' + band][use_cfhtls_wide]
    merr_megacam[use_wirds] = master_catalogue['merr_wirds_' + band][use_wirds]
    merr_megacam[use_cfhtlens] = master_catalogue['merr_cfhtlens_' + band][use_cfhtlens]
    merr_megacam[use_candels] = master_catalogue['merr_candels-megacam_' + band][use_candels]
    merr_megacam[use_irac] = master_catalogue['merr_irac-megacam_' + band][use_irac]

    flag_megacam = np.full(len(master_catalogue), False, dtype=bool)
    flag_megacam[use_cfhtls_deep] = master_catalogue['flag_cfhtls-deep_' + band][use_cfhtls_deep]
    flag_megacam[use_cfhtls_wide] = master_catalogue['flag_cfhtls-wide_' + band][use_cfhtls_wide]
    flag_megacam[use_wirds] = master_catalogue['flag_wirds_' + band][use_wirds]
    flag_megacam[use_cfhtlens] = master_catalogue['flag_cfhtlens_' + band][use_cfhtlens]
    flag_megacam[use_candels] = master_catalogue['flag_candels-megacam_' + band][use_candels]
    flag_megacam[use_irac] = master_catalogue['flag_irac-megacam_' + band][use_irac]

    master_catalogue.add_column(Column(data=f_megacam, name="f_megacam_" + band))
    master_catalogue.add_column(Column(data=ferr_megacam, name="ferr_megacam_" + band))
    master_catalogue.add_column(Column(data=m_megacam, name="m_megacam_" + band))
    master_catalogue.add_column(Column(data=merr_megacam, name="merr_megacam_" + band))
    master_catalogue.add_column(Column(data=flag_megacam, name="flag_megacam_" + band))

    old_columns = ['f_cfhtls-deep_' + band,
                    'ferr_cfhtls-deep_' + band,
                    'm_cfhtls-deep_' + band, 
                    'merr_cfhtls-deep_' + band,
                    'flag_cfhtls-deep_' + band,
                   'f_cfhtls-wide_' + band,
                    'ferr_cfhtls-wide_' + band,
                    'm_cfhtls-wide_' + band, 
                    'merr_cfhtls-wide_' + band,
                    'flag_cfhtls-wide_' + band,
                  'f_wirds_' + band,
                    'ferr_wirds_' + band,
                    'm_wirds_' + band, 
                    'merr_wirds_' + band,
                    'flag_wirds_' + band,
                  'f_cfhtlens_' + band,
                    'ferr_cfhtlens_' + band,
                    'm_cfhtlens_' + band, 
                    'merr_cfhtlens_' + band,
                    'flag_cfhtlens_' + band,
                  'f_candels-megacam_' + band,
                    'ferr_candels-megacam_' + band,
                    'm_candels-megacam_' + band, 
                    'merr_candels-megacam_' + band,
                    'flag_candels-megacam_' + band,
                  'f_irac-megacam_' + band,
                    'ferr_irac-megacam_' + band,
                    'm_irac-megacam_' + band, 
                    'merr_irac-megacam_' + band,
                    'flag_irac-megacam_' + band,]

    
    master_catalogue.remove_columns(old_columns)

    origin = np.full(len(master_catalogue), '     ', dtype='<U5')
    origin[use_cfhtls_deep] = "CFHTLS-DEEP"
    origin[use_cfhtls_wide] = "CFHTLS-WIDE"
    origin[use_wirds] = "CFHT-WIRDS"
    origin[use_cfhtlens] = "CFHTLenS"
    origin[use_candels] = "CANDELS"
    origin[use_irac] = "IRAC-EGS"
    
    megacam_origin.add_column(Column(data=origin, name= 'f_megacam_' + band ))
    
    # Megacam aperture flux 
    has_ap_cfhtls_deep = ~np.isnan(master_catalogue['f_ap_cfhtls-deep_' + band])
    has_ap_cfhtls_wide = ~np.isnan(master_catalogue['f_ap_cfhtls-wide_' + band])
    has_ap_wirds = ~np.isnan(master_catalogue['f_ap_wirds_' + band])
    has_ap_cfhtlens = ~np.isnan(master_catalogue['f_ap_cfhtlens_' + band])
    has_ap_candels = ~np.isnan(master_catalogue['f_ap_candels-megacam_' + band])
    
    use_ap_cfhtls_deep = has_ap_cfhtls_deep
    use_ap_cfhtls_wide = has_ap_cfhtls_wide & ~has_ap_cfhtls_deep
    use_ap_wirds       = has_ap_wirds       & ~has_ap_cfhtls_deep & ~has_ap_cfhtls_wide
    use_ap_cfhtlens    = has_ap_cfhtlens    & ~has_ap_cfhtls_deep & ~has_ap_cfhtls_wide & ~has_ap_wirds   
    use_ap_candels     = has_ap_candels     & ~has_ap_cfhtls_deep & ~has_ap_cfhtls_wide & ~has_ap_wirds & ~has_ap_cfhtlens   

    f_ap_megacam = np.full(len(master_catalogue), np.nan)
    f_ap_megacam[use_ap_cfhtls_deep] = master_catalogue['f_ap_cfhtls-deep_' + band][use_ap_cfhtls_deep]
    f_ap_megacam[use_ap_cfhtls_wide] = master_catalogue['f_ap_cfhtls-wide_' + band][use_ap_cfhtls_wide]
    f_ap_megacam[use_ap_wirds] = master_catalogue['f_ap_wirds_' + band][use_ap_wirds]
    f_ap_megacam[use_ap_cfhtlens] = master_catalogue['f_ap_cfhtlens_' + band][use_ap_cfhtlens]
    f_ap_megacam[use_ap_candels] = master_catalogue['f_ap_candels-megacam_' + band][use_ap_candels]

    ferr_ap_megacam = np.full(len(master_catalogue), np.nan)
    ferr_ap_megacam[use_ap_cfhtls_deep] = master_catalogue['ferr_ap_cfhtls-deep_' + band][use_ap_cfhtls_deep]
    ferr_ap_megacam[use_ap_cfhtls_wide] = master_catalogue['ferr_ap_cfhtls-wide_' + band][use_ap_cfhtls_wide]
    ferr_ap_megacam[use_ap_wirds] = master_catalogue['ferr_ap_wirds_' + band][use_ap_wirds]
    ferr_ap_megacam[use_ap_cfhtlens] = master_catalogue['ferr_ap_cfhtlens_' + band][use_ap_cfhtlens]
    ferr_ap_megacam[use_ap_candels] = master_catalogue['ferr_ap_candels-megacam_' + band][use_ap_candels]
    
    m_ap_megacam = np.full(len(master_catalogue), np.nan)
    m_ap_megacam[use_ap_cfhtls_deep] = master_catalogue['m_ap_cfhtls-deep_' + band][use_ap_cfhtls_deep]
    m_ap_megacam[use_ap_cfhtls_wide] = master_catalogue['m_ap_cfhtls-wide_' + band][use_ap_cfhtls_wide]
    m_ap_megacam[use_ap_wirds] = master_catalogue['m_ap_wirds_' + band][use_ap_wirds]
    m_ap_megacam[use_ap_cfhtlens] = master_catalogue['m_ap_cfhtlens_' + band][use_ap_cfhtlens]
    m_ap_megacam[use_ap_candels] = master_catalogue['m_ap_candels-megacam_' + band][use_ap_candels]

    merr_ap_megacam = np.full(len(master_catalogue), np.nan)
    merr_ap_megacam[use_ap_cfhtls_deep] = master_catalogue['merr_ap_cfhtls-deep_' + band][use_ap_cfhtls_deep]
    merr_ap_megacam[use_ap_cfhtls_wide] = master_catalogue['merr_ap_cfhtls-wide_' + band][use_ap_cfhtls_wide]
    merr_ap_megacam[use_ap_wirds] = master_catalogue['merr_ap_wirds_' + band][use_ap_wirds]
    merr_ap_megacam[use_ap_cfhtlens] = master_catalogue['merr_ap_cfhtlens_' + band][use_ap_cfhtlens]
    merr_ap_megacam[use_ap_candels] = master_catalogue['merr_ap_candels-megacam_' + band][use_ap_candels]

    master_catalogue.add_column(Column(data=f_ap_megacam, name="f_ap_megacam_" + band))
    master_catalogue.add_column(Column(data=ferr_ap_megacam, name="ferr_ap_megacam_" + band))
    master_catalogue.add_column(Column(data=m_ap_megacam, name="m_ap_megacam_" + band))
    master_catalogue.add_column(Column(data=merr_ap_megacam, name="merr_ap_megacam_" + band))


    old_ap_columns = ['f_ap_cfhtls-deep_' + band,
                    'ferr_ap_cfhtls-deep_' + band,
                    'm_ap_cfhtls-deep_' + band, 
                    'merr_ap_cfhtls-deep_' + band,
                  'f_ap_cfhtls-wide_' + band,
                    'ferr_ap_cfhtls-wide_' + band,
                    'm_ap_cfhtls-wide_' + band, 
                    'merr_ap_cfhtls-wide_' + band,
                  'f_ap_wirds_' + band,
                    'ferr_ap_wirds_' + band,
                    'm_ap_wirds_' + band, 
                    'merr_ap_wirds_' + band,
                  'f_ap_cfhtlens_' + band,
                    'ferr_ap_cfhtlens_' + band,
                    'm_ap_cfhtlens_' + band, 
                    'merr_ap_cfhtlens_' + band,
                  'f_ap_candels-megacam_' + band,
                    'ferr_ap_candels-megacam_' + band,
                    'm_ap_candels-megacam_' + band, 
                    'merr_ap_candels-megacam_' + band]

    
    master_catalogue.remove_columns(old_ap_columns)

    origin_ap = np.full(len(master_catalogue), '     ', dtype='<U5')
    origin_ap[use_ap_cfhtls_deep] = "CFHTLS-DEEP"
    origin_ap[use_ap_cfhtls_wide] = "CFHTLS-WIDE"
    origin_ap[use_ap_wirds] = "CFHT-WIRDS"
    origin_ap[use_ap_cfhtlens] = "CFHTLenS"
    origin_ap[use_ap_candels] = "CANDELS"
    
    megacam_origin.add_column(Column(data=origin_ap, name= 'f_ap_megacam_' + band ))
    

   
    megacam_stats['CFHTLS-DEEP'][megacam_stats['Band'] == band] = np.sum(has_cfhtls_deep)
    megacam_stats['CFHTLS-WIDE'][megacam_stats['Band'] == band] = np.sum(has_cfhtls_wide)
    megacam_stats['CFHT-WIRDS'][megacam_stats['Band'] == band] = np.sum(has_wirds)
    megacam_stats['CFHTLenS'][megacam_stats['Band'] == band] = np.sum(has_cfhtlens)
    megacam_stats['CANDELS'][megacam_stats['Band'] == band] = np.sum(has_candels)
    megacam_stats['use CFHTLS-DEEP'][megacam_stats['Band'] == band] = np.sum(use_cfhtls_deep)
    megacam_stats['use CFHTLS-WIDE'][megacam_stats['Band'] == band] = np.sum(use_cfhtls_wide)
    megacam_stats['use CFHT-WIRDS'][megacam_stats['Band'] == band] = np.sum(use_wirds)
    megacam_stats['use CFHTLenS'][megacam_stats['Band'] == band] = np.sum(use_cfhtlens)
    megacam_stats['use CANDELS'][megacam_stats['Band'] == band] = np.sum(use_candels)    
    megacam_stats['CFHTLS-DEEP ap'][megacam_stats['Band'] == band] = np.sum(has_ap_cfhtls_deep)
    megacam_stats['CFHTLS-WIDE ap'][megacam_stats['Band'] == band] = np.sum(has_ap_cfhtls_wide)
    megacam_stats['CFHT-WIRDS ap'][megacam_stats['Band'] == band] = np.sum(has_ap_wirds)
    megacam_stats['CFHTLenS ap'][megacam_stats['Band'] == band] = np.sum(has_ap_cfhtlens)
    megacam_stats['CANDELS ap'][megacam_stats['Band'] == band] = np.sum(has_ap_candels)
    megacam_stats['use CFHTLS-DEEP ap'][megacam_stats['Band'] == band] = np.sum(use_ap_cfhtls_deep)
    megacam_stats['use CFHTLS-WIDE ap'][megacam_stats['Band'] == band] = np.sum(use_ap_cfhtls_wide)
    megacam_stats['use CFHT-WIRDS ap'][megacam_stats['Band'] == band] = np.sum(use_ap_wirds)
    megacam_stats['use CFHTLenS ap'][megacam_stats['Band'] == band] = np.sum(use_ap_cfhtlens)
    megacam_stats['use CANDELS ap'][megacam_stats['Band'] == band] = np.sum(use_ap_candels)

In [ ]:
megacam_stats.show_in_notebook()

In [ ]:
megacam_origin.write("{}/egs_megacam_fluxes_origins{}.fits".format(OUT_DIR, SUFFIX), overwrite=True)

## CFHT WIRCAM

We have WIRCAM J,H, and Ks from both CFHT-WIRDS (Ks prior and blind) and CANDELS-EGS. Since the CANDELS will have very deep priors the WIRCAM fluxes are worth keepting to constrain photo-z. We therefor take the CFHT-WIRDS fluxes if they are there but keep all the CANDELS fluxes for sources that only have those.

In [ ]:
wircam_origin = Table()
wircam_origin.add_column(master_catalogue['help_id'])

In [ ]:
wircam_stats = Table()
wircam_stats.add_column(Column(data=['j', 'h', 'k'], name="Band"))
for col in ["CFHT-WIRDS", "CANDELS"]:
    wircam_stats.add_column(Column(data=np.full(3, 0), name="{}".format(col)))
    wircam_stats.add_column(Column(data=np.full(3, 0), name="use {}".format(col)))
    wircam_stats.add_column(Column(data=np.full(3, 0), name="{} ap".format(col)))
    wircam_stats.add_column(Column(data=np.full(3, 0), name="use {} ap".format(col)))

In [ ]:
wircam_bands = ['j', 'h', 'k'] 
for band in wircam_bands:

    # wircam total flux 
    has_wirds = ~np.isnan(master_catalogue['f_wirds_' + band])
    has_candels = ~np.isnan(master_catalogue['f_candels-wircam_' + band])
    
    use_wirds =       has_wirds        
    use_candels =     has_candels & ~has_wirds 

    f_wircam = np.full(len(master_catalogue), np.nan)
    f_wircam[use_wirds] = master_catalogue['f_wirds_' + band][use_wirds]
    f_wircam[use_candels] = master_catalogue['f_candels-wircam_' + band][use_candels]

    ferr_wircam = np.full(len(master_catalogue), np.nan)
    ferr_wircam[use_wirds] = master_catalogue['ferr_wirds_' + band][use_wirds]
    ferr_wircam[use_candels] = master_catalogue['ferr_candels-wircam_' + band][use_candels]
    
    m_wircam = np.full(len(master_catalogue), np.nan)
    m_wircam[use_wirds] = master_catalogue['m_wirds_' + band][use_wirds]
    m_wircam[use_candels] = master_catalogue['m_candels-wircam_' + band][use_candels]

    merr_wircam = np.full(len(master_catalogue), np.nan)
    merr_wircam[use_wirds] = master_catalogue['merr_wirds_' + band][use_wirds]
    merr_wircam[use_candels] = master_catalogue['merr_candels-wircam_' + band][use_candels]

    flag_wircam = np.full(len(master_catalogue), False, dtype=bool)
    flag_wircam[use_wirds] = master_catalogue['flag_wirds_' + band][use_wirds]
    flag_wircam[use_candels] = master_catalogue['flag_candels-wircam_' + band][use_candels]

    master_catalogue.add_column(Column(data=f_wircam, name="f_wircam_" + band))
    master_catalogue.add_column(Column(data=ferr_wircam, name="ferr_wircam_" + band))
    master_catalogue.add_column(Column(data=m_wircam, name="m_wircam_" + band))
    master_catalogue.add_column(Column(data=merr_wircam, name="merr_wircam_" + band))
    master_catalogue.add_column(Column(data=flag_wircam, name="flag_wircam_" + band))

    old_columns = ['f_wirds_' + band,
                    'ferr_wirds_' + band,
                    'm_wirds_' + band, 
                    'merr_wirds_' + band,
                    'flag_wirds_' + band,
                  'f_candels-wircam_' + band,
                    'ferr_candels-wircam_' + band,
                    'm_candels-wircam_' + band, 
                    'merr_candels-wircam_' + band,
                    'flag_candels-wircam_' + band,]

    
    master_catalogue.remove_columns(old_columns)

    origin = np.full(len(master_catalogue), '     ', dtype='<U5')
    origin[use_wirds] = "CFHT-WIRDS"
    origin[use_candels] = "CANDELS"
    
    wircam_origin.add_column(Column(data=origin, name= 'f_wircam_' + band ))
    
    # wircam aperture flux 
    has_ap_wirds = ~np.isnan(master_catalogue['f_ap_wirds_' + band])
    has_ap_candels = ~np.isnan(master_catalogue['f_ap_candels-wircam_' + band])
    
    use_ap_wirds       = has_ap_wirds      
    use_ap_candels     = has_ap_candels     & ~has_ap_wirds  

    f_ap_wircam = np.full(len(master_catalogue), np.nan)
    f_ap_wircam[use_ap_wirds] = master_catalogue['f_ap_wirds_' + band][use_ap_wirds]
    f_ap_wircam[use_ap_candels] = master_catalogue['f_ap_candels-wircam_' + band][use_ap_candels]

    ferr_ap_wircam = np.full(len(master_catalogue), np.nan)
    ferr_ap_wircam[use_ap_wirds] = master_catalogue['ferr_ap_wirds_' + band][use_ap_wirds]
    ferr_ap_wircam[use_ap_candels] = master_catalogue['ferr_ap_candels-wircam_' + band][use_ap_candels]
    
    m_ap_wircam = np.full(len(master_catalogue), np.nan)
    m_ap_wircam[use_ap_wirds] = master_catalogue['m_ap_wirds_' + band][use_ap_wirds]
    m_ap_wircam[use_ap_candels] = master_catalogue['m_ap_candels-wircam_' + band][use_ap_candels]

    merr_ap_wircam = np.full(len(master_catalogue), np.nan)
    merr_ap_wircam[use_ap_wirds] = master_catalogue['merr_ap_wirds_' + band][use_ap_wirds]
    merr_ap_wircam[use_ap_candels] = master_catalogue['merr_ap_candels-wircam_' + band][use_ap_candels]

    master_catalogue.add_column(Column(data=f_ap_wircam, name="f_ap_wircam_" + band))
    master_catalogue.add_column(Column(data=ferr_ap_wircam, name="ferr_ap_wircam_" + band))
    master_catalogue.add_column(Column(data=m_ap_wircam, name="m_ap_wircam_" + band))
    master_catalogue.add_column(Column(data=merr_ap_wircam, name="merr_ap_wircam_" + band))


    old_ap_columns = ['f_ap_wirds_' + band,
                    'ferr_ap_wirds_' + band,
                    'm_ap_wirds_' + band, 
                    'merr_ap_wirds_' + band,
                  'f_ap_candels-wircam_' + band,
                    'ferr_ap_candels-wircam_' + band,
                    'm_ap_candels-wircam_' + band, 
                    'merr_ap_candels-wircam_' + band]

    
    master_catalogue.remove_columns(old_ap_columns)

    origin_ap = np.full(len(master_catalogue), '     ', dtype='<U5')
    origin_ap[use_ap_wirds] = "CFHT-WIRDS"
    origin_ap[use_ap_candels] = "CANDELS"
    
    wircam_origin.add_column(Column(data=origin_ap, name= 'f_ap_wircam_' + band ))
    

   

    wircam_stats['CFHT-WIRDS'][wircam_stats['Band'] == band] = np.sum(has_wirds)
    wircam_stats['CANDELS'][wircam_stats['Band'] == band] = np.sum(has_candels)
    wircam_stats['use CFHT-WIRDS'][wircam_stats['Band'] == band] = np.sum(use_wirds)
    wircam_stats['use CANDELS'][wircam_stats['Band'] == band] = np.sum(use_candels)    
    wircam_stats['CFHT-WIRDS ap'][wircam_stats['Band'] == band] = np.sum(has_ap_wirds)
    wircam_stats['CANDELS ap'][wircam_stats['Band'] == band] = np.sum(has_ap_candels)
    wircam_stats['use CFHT-WIRDS ap'][wircam_stats['Band'] == band] = np.sum(use_ap_wirds)
    wircam_stats['use CANDELS ap'][wircam_stats['Band'] == band] = np.sum(use_ap_candels)

In [ ]:
wircam_stats.show_in_notebook()

In [ ]:
wircam_origin.write("{}/egs_wircam_fluxes_origins{}.fits".format(OUT_DIR, SUFFIX), overwrite=True)

### IRAC

We have IRAC from the IRAC-EGS catalogue and from CANDELS. We take the CANDELS fluxes preferentially.

In [ ]:
irac_origin = Table()
irac_origin.add_column(master_catalogue['help_id'])

In [ ]:
irac_stats = Table()
irac_stats.add_column(Column(data=['i1', 'i2', 'i3', 'i4'], name="Band"))
for col in ["CANDELS", "IRAC-EGS"]:
    irac_stats.add_column(Column(data=np.full(4, 0), name="{}".format(col)))
    irac_stats.add_column(Column(data=np.full(4, 0), name="use {}".format(col)))
    irac_stats.add_column(Column(data=np.full(4, 0), name="{} ap".format(col)))
    irac_stats.add_column(Column(data=np.full(4, 0), name="use {} ap".format(col)))

In [ ]:
irac_bands = ['i1', 'i2', 'i3', 'i4'] 
for band in irac_bands:

    # IRAC total flux 
    has_candels = ~np.isnan(master_catalogue['f_candels-irac_' + band])
    has_irac = ~np.isnan(master_catalogue['f_irac-egs_' + band])
    
    
    use_candels =       has_candels       
    use_irac =     has_irac & ~has_candels

    f_irac = np.full(len(master_catalogue), np.nan)
    f_irac[use_candels] = master_catalogue['f_candels-irac_' + band][use_candels]
    f_irac[use_irac] = master_catalogue['f_irac-egs_' + band][use_irac]
    
    ferr_irac = np.full(len(master_catalogue), np.nan)
    ferr_irac[use_candels] = master_catalogue['ferr_candels-irac_' + band][use_candels]
    ferr_irac[use_irac] = master_catalogue['ferr_irac-egs_' + band][use_irac]
    
    m_irac = np.full(len(master_catalogue), np.nan)
    m_irac[use_candels] = master_catalogue['m_candels-irac_' + band][use_candels]
    m_irac[use_irac] = master_catalogue['m_irac-egs_' + band][use_irac]
    
    merr_irac = np.full(len(master_catalogue), np.nan)
    merr_irac[use_candels] = master_catalogue['merr_candels-irac_' + band][use_candels]
    merr_irac[use_irac] = master_catalogue['merr_irac-egs_' + band][use_irac]
    
    flag_irac = np.full(len(master_catalogue), np.nan)
    flag_irac[use_candels] = master_catalogue['flag_candels-irac_' + band][use_candels]
    flag_irac[use_irac] = master_catalogue['flag_irac-egs_' + band][use_irac]



    master_catalogue.add_column(Column(data=f_irac, name="f_irac_" + band))
    master_catalogue.add_column(Column(data=ferr_irac, name="ferr_irac_" + band))
    master_catalogue.add_column(Column(data=m_irac, name="m_irac_" + band))
    master_catalogue.add_column(Column(data=merr_irac, name="merr_irac_" + band))
    master_catalogue.add_column(Column(data=flag_irac, name="flag_irac_" + band))

    old_columns = ['f_candels-irac_' + band,
                    'ferr_candels-irac_' + band,
                    'm_candels-irac_' + band, 
                    'merr_candels-irac_' + band,
                    'flag_candels-irac_' + band,
                  'f_irac-egs_' + band,
                    'ferr_irac-egs_' + band,
                    'm_irac-egs_' + band, 
                    'merr_irac-egs_' + band,
                    'flag_irac-egs_' + band,]

    
    master_catalogue.remove_columns(old_columns)

    origin = np.full(len(master_catalogue), '     ', dtype='<U5')
    origin[use_candels] = "CANDELS"
    origin[use_irac] = "IRAC-EGS"
    
    irac_origin.add_column(Column(data=origin, name= 'f_irac_' + band ))
    
  
 
    

   

    irac_stats['CANDELS'][irac_stats['Band'] == band] = np.sum(has_candels)
    irac_stats['IRAC-EGS'][irac_stats['Band'] == band] = np.sum(has_irac)
    irac_stats['use CANDELS'][irac_stats['Band'] == band] = np.sum(use_candels)
    irac_stats['use IRAC-EGS'][irac_stats['Band'] == band] = np.sum(use_irac)    


In [ ]:
irac_stats.show_in_notebook()

In [ ]:
irac_origin.write("{}/egs_irac_fluxes_origins{}.fits".format(OUT_DIR, SUFFIX), overwrite=True)

### AEGIS
We have AEGIS (WIRCS instrument on Palomar telescope) data from the AEGIS catalogue and from IRAC-EGS. We take the AEGIS fluxes preferentially 

In [ ]:
aegis_origin = Table()
aegis_origin.add_column(master_catalogue['help_id'])

In [ ]:
aegis_stats = Table()
aegis_stats.add_column(Column(data=['j',  'k'], name="Band"))
for col in ["AEGIS", "IRAC-EGS"]:
    aegis_stats.add_column(Column(data=np.full(2, 0), name="{}".format(col)))
    aegis_stats.add_column(Column(data=np.full(2, 0), name="use {}".format(col)))
    aegis_stats.add_column(Column(data=np.full(2, 0), name="{} ap".format(col)))
    aegis_stats.add_column(Column(data=np.full(2, 0), name="use {} ap".format(col)))

In [ ]:
aegis_bands = ['j',  'k'] 
for band in aegis_bands:

    # wircam total flux 
    has_aegis = ~np.isnan(master_catalogue['f_aegis_' + band])
    has_irac = ~np.isnan(master_catalogue['f_irac-aegis_' + band])
    
    use_aegis =       has_aegis        
    use_irac =     has_irac & ~has_aegis

    f_aegis = np.full(len(master_catalogue), np.nan)
    f_aegis[use_aegis] = master_catalogue['f_aegis_' + band][use_aegis]
    f_aegis[use_irac] = master_catalogue['f_irac-aegis_' + band][use_irac]

    ferr_aegis = np.full(len(master_catalogue), np.nan)
    ferr_aegis[use_aegis] = master_catalogue['ferr_aegis_' + band][use_aegis]
    ferr_aegis[use_irac] = master_catalogue['ferr_irac-aegis_' + band][use_irac]
    
    m_aegis = np.full(len(master_catalogue), np.nan)
    m_aegis[use_aegis] = master_catalogue['m_aegis_' + band][use_aegis]
    m_aegis[use_irac] = master_catalogue['m_irac-aegis_' + band][use_irac]
    
    merr_aegis = np.full(len(master_catalogue), np.nan)
    merr_aegis[use_aegis] = master_catalogue['merr_aegis_' + band][use_aegis]
    merr_aegis[use_irac] = master_catalogue['merr_irac-aegis_' + band][use_irac]
    
    flag_aegis = np.full(len(master_catalogue), np.nan)
    flag_aegis[use_aegis] = master_catalogue['flag_aegis_' + band][use_aegis]
    flag_aegis[use_irac] = master_catalogue['flag_irac-aegis_' + band][use_irac]

    master_catalogue.add_column(Column(data=f_aegis, name="f_wircs_" + band))
    master_catalogue.add_column(Column(data=ferr_aegis, name="ferr_wircs_" + band))
    master_catalogue.add_column(Column(data=m_aegis, name="m_wircs_" + band))
    master_catalogue.add_column(Column(data=merr_aegis, name="merr_wircs_" + band))
    master_catalogue.add_column(Column(data=flag_aegis, name="flag_wircs_" + band))

    old_columns = ['f_aegis_' + band,
                    'ferr_aegis_' + band,
                    'm_aegis_' + band, 
                    'merr_aegis_' + band,
                    'flag_aegis_' + band,
                  'f_irac-aegis_' + band,
                    'ferr_irac-aegis_' + band,
                    'm_irac-aegis_' + band, 
                    'merr_irac-aegis_' + band,
                    'flag_irac-aegis_' + band,]

    
    master_catalogue.remove_columns(old_columns)

    origin = np.full(len(master_catalogue), '     ', dtype='<U5')
    origin[use_aegis] = "AEGIS"
    origin[use_irac] = "IRAC-EGS"
    
    aegis_origin.add_column(Column(data=origin, name= 'f_wircs_' + band ))
    
    # wircam aperture flux 
    if band == 'k':
        
        has_ap_aegis = ~np.isnan(master_catalogue['f_ap_aegis_' + band])
        
        master_catalogue.rename_column('f_ap_aegis_k', 'f_ap_wircs_k')
        master_catalogue.rename_column('ferr_ap_aegis_k', 'ferr_ap_wircs_k')
        master_catalogue.rename_column('m_ap_aegis_k', 'm_ap_wircs_k')
        master_catalogue.rename_column('merr_ap_aegis_k', 'merr_ap_wircs_k')
        
        origin_ap = np.full(len(master_catalogue), '     ', dtype='<U5')
        origin_ap[use_ap_aegis] = "AEGIS"
       
    
        aegis_origin.add_column(Column(data=origin_ap, name= 'f_ap_wircs_' + band ))
        aegis_stats['AEGIS ap'][wircam_stats['Band'] == band] = np.sum(has_ap_aegis)
        
        aegis_stats['use AEGIS ap'][wircam_stats['Band'] == band] = np.sum(use_ap_aegis)
        
    

   

    aegis_stats['AEGIS'][aegis_stats['Band'] == band] = np.sum(has_aegis)
    aegis_stats['IRAC-EGS'][aegis_stats['Band'] == band] = np.sum(has_irac)
    aegis_stats['use AEGIS'][aegis_stats['Band'] == band] = np.sum(use_aegis)
    aegis_stats['use IRAC-EGS'][aegis_stats['Band'] == band] = np.sum(use_irac)    
    

In [ ]:
aegis_stats.show_in_notebook()

In [ ]:
aegis_origin.write("{}/egs_aegis_fluxes_origins{}.fits".format(OUT_DIR, SUFFIX), overwrite=True)

### HST: CANDELS vs IRAC-EGS
We take CANDELS over IRAC

In [ ]:
hst_origin = Table()
hst_origin.add_column(master_catalogue['help_id'])

In [ ]:
hst_stats = Table()
hst_stats.add_column(Column(data=['f814w',  'f606w'], name="Band"))
for col in ["CANDELS", "IRAC-EGS"]:
    hst_stats.add_column(Column(data=np.full(2, 0), name="{}".format(col)))
    hst_stats.add_column(Column(data=np.full(2, 0), name="use {}".format(col)))
    hst_stats.add_column(Column(data=np.full(2, 0), name="{} ap".format(col)))
    hst_stats.add_column(Column(data=np.full(2, 0), name="use {} ap".format(col)))

In [ ]:
hst_bands = ['f814w',  'f606w']

for band in hst_bands:

    # HST total flux 
    has_candels = ~np.isnan(master_catalogue['f_acs_' + band])
    has_irac = ~np.isnan(master_catalogue['f_irac-acs_' + band])
    
    use_candels =       has_candels        
    use_irac =     has_irac & ~has_candels

    master_catalogue['f_acs_' + band][use_irac] = master_catalogue['f_irac-acs_' + band][use_irac]
    master_catalogue['ferr_acs_' + band][use_irac] = master_catalogue['ferr_irac-acs_' + band][use_irac]
    master_catalogue['m_acs_' + band][use_irac] = master_catalogue['m_irac-acs_' + band][use_irac]
    master_catalogue['merr_acs_' + band][use_irac] = master_catalogue['merr_irac-acs_' + band][use_irac]
    master_catalogue['flag_acs_' + band][use_irac] = master_catalogue['flag_irac-acs_' + band][use_irac]

    old_columns = ['f_irac-acs_' + band,
                    'ferr_irac-acs_' + band,
                    'm_irac-acs_' + band, 
                    'merr_irac-acs_' + band,
                    'flag_irac-acs_' + band,]

    
    master_catalogue.remove_columns(old_columns)

    origin = np.full(len(master_catalogue), '     ', dtype='<U5')
    origin[use_candels] = "CANDELS"
    origin[use_irac] = "IRAC-EGS"
    
    hst_origin.add_column(Column(data=origin, name= 'f_cfht12k_' + band ))
    

        
    

   

    hst_stats['CANDELS'][hst_stats['Band'] == band] = np.sum(has_candels)
    hst_stats['IRAC-EGS'][hst_stats['Band'] == band] = np.sum(has_irac)
    hst_stats['use CANDELS'][hst_stats['Band'] == band] = np.sum(use_candels)
    hst_stats['use IRAC-EGS'][hst_stats['Band'] == band] = np.sum(use_irac)    
    

In [ ]:
hst_stats.show_in_notebook()

In [ ]:
hst_origin.write("{}/egs_hst_fluxes_origins{}.fits".format(OUT_DIR, SUFFIX), overwrite=True)

### CFHT12k : DEEP2 vs IRAC-EGS
We take DEEP2 preferentially

In [ ]:
cfht12k_origin = Table()
cfht12k_origin.add_column(master_catalogue['help_id'])

In [ ]:
cfht12k_stats = Table()
cfht12k_stats.add_column(Column(data=['b',  'r', 'i'], name="Band"))
for col in ["DEEP2", "IRAC-EGS"]:
    cfht12k_stats.add_column(Column(data=np.full(3, 0), name="{}".format(col)))
    cfht12k_stats.add_column(Column(data=np.full(3, 0), name="use {}".format(col)))
    cfht12k_stats.add_column(Column(data=np.full(3, 0), name="{} ap".format(col)))
    cfht12k_stats.add_column(Column(data=np.full(3, 0), name="use {} ap".format(col)))

In [ ]:
cfht12k_bands = ['b', 'r', 'i']

for band in cfht12k_bands:

    # cfht12k total flux 
    has_deep2 = ~np.isnan(master_catalogue['f_deep2_' + band])
    has_irac = ~np.isnan(master_catalogue['f_cfht12k_' + band])
    
    use_deep2 =       has_deep2        
    use_irac =     has_irac & ~has_deep2

    master_catalogue['f_cfht12k_' + band][use_deep2] = master_catalogue['f_deep2_' + band][use_deep2]
    master_catalogue['ferr_cfht12k_' + band][use_deep2] = master_catalogue['ferr_deep2_' + band][use_deep2]
    master_catalogue['m_cfht12k_' + band][use_deep2] = master_catalogue['m_deep2_' + band][use_deep2]
    master_catalogue['merr_cfht12k_' + band][use_deep2] = master_catalogue['merr_deep2_' + band][use_deep2]
    master_catalogue['flag_cfht12k_' + band][use_deep2] = master_catalogue['flag_deep2_' + band][use_deep2]

    old_columns = ['f_deep2_' + band,
                    'ferr_deep2_' + band,
                    'm_deep2_' + band, 
                    'merr_deep2_' + band,
                    'flag_deep2_' + band,]

    
    master_catalogue.remove_columns(old_columns)

    origin = np.full(len(master_catalogue), '     ', dtype='<U5')
    origin[use_deep2] = "DEEP2"
    origin[use_irac] = "IRAC-EGS"
    
    cfht12k_origin.add_column(Column(data=origin, name= 'f_cfht12k_' + band ))
    

        
    

   

    cfht12k_stats['DEEP2'][cfht12k_stats['Band'] == band] = np.sum(has_deep2)
    cfht12k_stats['IRAC-EGS'][cfht12k_stats['Band'] == band] = np.sum(has_irac)
    cfht12k_stats['use DEEP2'][cfht12k_stats['Band'] == band] = np.sum(use_deep2)
    cfht12k_stats['use IRAC-EGS'][cfht12k_stats['Band'] == band] = np.sum(use_irac)    
    

In [ ]:
cfht12k_stats.show_in_notebook()

In [ ]:
cfht12k_origin.write("{}/egs_cfht12k_fluxes_origins{}.fits".format(OUT_DIR, SUFFIX), overwrite=True)

## VIII.a Wavelength domain coverage

We add a binary `flag_optnir_obs` indicating that a source was observed in a given wavelength domain:

- 1 for observation in optical;
- 2 for observation in near-infrared;
- 4 for observation in mid-infrared (IRAC).

It's an integer binary flag, so a source observed both in optical and near-infrared by not in mid-infrared would have this flag at 1 + 2 = 3.

*Note 1: The observation flag is based on the creation of multi-order coverage maps from the catalogues, this may not be accurate, especially on the edges of the coverage.*

*Note 2: Being on the observation coverage does not mean having fluxes in that wavelength domain. For sources observed in one domain but having no flux in it, one must take into consideration de different depths in the catalogue we are using.*

In [ ]:
aegis_moc =  MOC(filename="../../dmu0/dmu0_AEGIS/data/EGS_Palomar_20160804_MOC.fits")
candels_moc =  MOC(filename="../../dmu0/dmu0_CANDELS-EGS/data/hlsp_candels_hst_wfc3_egs-tot-multiband_f160w_v1_MOC.fits")
wirds_moc =  MOC(filename="../../dmu0/dmu0_CFHT-WIRDS/data/EGS_Ks-priors_MOC.fits")
cfhtls_deep_moc =  MOC(filename="../../dmu0/dmu0_CFHTLS/data/CFHTLS-DEEP_EGS_MOC.fits")
cfhtls_wide_moc =  MOC(filename="../../dmu0/dmu0_CFHTLS/data/CFHTLS-WIDE_EGS_MOC.fits")
cfhtlens_moc =  MOC(filename="../../dmu0/dmu0_CFHTLenS/data/CFHTLenS_EGS_MOC.fits")
deep_moc =  MOC(filename="../../dmu0/dmu0_DEEP2/data/DEEP2_EGS_MOC.fits")
irac_moc =  MOC(filename="../../dmu0/dmu0_IRAC-EGS/data/IRAC_EGS_MOC.fits")
hsc_moc =  MOC(filename="../../dmu0/dmu0_HSC/data/HSC-PDR1_wide_EGS_MOC.fits")
ps1_moc =  MOC(filename="../../dmu0/dmu0_PanSTARRS1-3SS/data/PanSTARRS1-3SS_EGS_MOC.fits")
legacy_moc =  MOC(filename="../../dmu0/dmu0_LegacySurvey/data/LegacySurvey-dr4_EGS_MOC.fits")
uhs_moc =  MOC(filename="../../dmu0/dmu0_UHS/data/UHS-DR1_EGS_MOC.fits")

In [ ]:
was_observed_optical = inMoc(
    master_catalogue['ra'], master_catalogue['dec'],
    aegis_moc + candels_moc + cfhtls_deep_moc + cfhtls_wide_moc + cfhtlens_moc + deep_moc + hsc_moc + ps1_moc + legacy_moc) 

was_observed_nir = inMoc(
    master_catalogue['ra'], master_catalogue['dec'],
    wirds_moc + uhs_moc 
)

was_observed_mir = inMoc(
    master_catalogue['ra'], master_catalogue['dec'],
    irac_moc
)

In [ ]:
master_catalogue.add_column(
    Column(
        1 * was_observed_optical + 2 * was_observed_nir + 4 * was_observed_mir,
        name="flag_optnir_obs")
)

## VIII.b Wavelength domain detection

We add a binary `flag_optnir_det` indicating that a source was detected in a given wavelength domain:

- 1 for detection in optical;
- 2 for detection in near-infrared;
- 4 for detection in mid-infrared (IRAC).

It's an integer binary flag, so a source detected both in optical and near-infrared by not in mid-infrared would have this flag at 1 + 2 = 3.

*Note 1: We use the total flux columns to know if the source has flux, in some catalogues, we may have aperture flux and no total flux.*

To get rid of artefacts (chip edges, star flares, etc.) we consider that a source is detected in one wavelength domain when it has a flux value in **at least two bands**. That means that good sources will be excluded from this flag when they are on the coverage of only one band.

In [ ]:
#Rename UHS to UKIDSS
for col in master_catalogue.colnames:
    if col.endswith('_uhs_j'):
        master_catalogue.rename_column(col, col.replace('_uhs_j', '_ukidss_j'))
    if col.endswith('_wircam_k'):
        master_catalogue.rename_column(col, col.replace('_wircam_k', '_wircam_ks'))

In [ ]:

nb_optical_flux = (
#HSC
    1 * ~np.isnan(master_catalogue['f_suprime_g']) +
    1 * ~np.isnan(master_catalogue['f_suprime_r']) +
    1 * ~np.isnan(master_catalogue['f_suprime_i']) +
    1 * ~np.isnan(master_catalogue['f_suprime_z']) +
    1 * ~np.isnan(master_catalogue['f_suprime_y']) +
#PS1
    1 * ~np.isnan(master_catalogue['f_gpc1_g']) +
    1 * ~np.isnan(master_catalogue['f_gpc1_r']) +
    1 * ~np.isnan(master_catalogue['f_gpc1_i']) +
    1 * ~np.isnan(master_catalogue['f_gpc1_z']) +
    1 * ~np.isnan(master_catalogue['f_gpc1_y']) +
#CFHT
    1 * ~np.isnan(master_catalogue['f_megacam_u']) +
    1 * ~np.isnan(master_catalogue['f_megacam_g']) +
    1 * ~np.isnan(master_catalogue['f_megacam_r']) +
    1 * ~np.isnan(master_catalogue['f_megacam_i']) +
    1 * ~np.isnan(master_catalogue['f_megacam_z']) 

)

nb_nir_flux = (
    1 * ~np.isnan(master_catalogue['f_ukidss_j']) +
    1 * ~np.isnan(master_catalogue['f_wircam_j']) +
    1 * ~np.isnan(master_catalogue['f_wircam_h']) +
    1 * ~np.isnan(master_catalogue['f_wircam_ks']) 
    
)

nb_mir_flux = (
    1 * ~np.isnan(master_catalogue['f_irac_i1']) +
    1 * ~np.isnan(master_catalogue['f_irac_i2']) +
    1 * ~np.isnan(master_catalogue['f_irac_i3']) +
    1 * ~np.isnan(master_catalogue['f_irac_i4'])
)

In [ ]:
has_optical_flux = nb_optical_flux >= 2
has_nir_flux = nb_nir_flux >= 2
has_mir_flux = nb_mir_flux >= 2

master_catalogue.add_column(
    Column(
        1 * has_optical_flux + 2 * has_nir_flux + 4 * has_mir_flux,
        name="flag_optnir_det")
)

## IX - Cross-identification table

We are producing a table associating to each HELP identifier, the identifiers of the sources in the pristine catalogues. This can be used to easily get additional information from them.

For convenience, we also cross-match the master list with the SDSS catalogue and add the objID associated with each source, if any. **TODO: should we correct the astrometry with respect to Gaia positions?**

In [ ]:
#
# Addind SDSS ids
#
sdss = Table.read("../../dmu0/dmu0_SDSS-DR13/data/SDSS-DR13_EGS.fits")['objID', 'ra', 'dec']
sdss_coords = SkyCoord(sdss['ra'] * u.deg, sdss['dec'] * u.deg)
idx_ml, d2d, _ = sdss_coords.match_to_catalog_sky(SkyCoord(master_catalogue['ra'], master_catalogue['dec']))
idx_sdss = np.arange(len(sdss))

# Limit the cross-match to 1 arcsec
mask = d2d <= 1. * u.arcsec
idx_ml = idx_ml[mask]
idx_sdss = idx_sdss[mask]
d2d = d2d[mask]
nb_orig_matches = len(idx_ml)

# In case of multiple associations of one master list object to an SDSS object, we keep only the
# association to the nearest one.
sort_idx = np.argsort(d2d)
idx_ml = idx_ml[sort_idx]
idx_sdss = idx_sdss[sort_idx]
_, unique_idx = np.unique(idx_ml, return_index=True)
idx_ml = idx_ml[unique_idx]
idx_sdss = idx_sdss[unique_idx]
print("{} master list rows had multiple associations.".format(nb_orig_matches - len(idx_ml)))

# Adding the ObjID to the master list
master_catalogue.add_column(Column(data=np.full(len(master_catalogue), -1, dtype='>i8'), name="sdss_id"))
master_catalogue['sdss_id'][idx_ml] = sdss['objID'][idx_sdss]

In [ ]:

id_names = []
for col in master_catalogue.colnames:
    if '_id' in col:
        id_names += [col]
    if '_intid' in col:
        id_names += [col]
        
print(id_names)

In [ ]:
master_catalogue[id_names].write(
    "{}/master_list_cross_ident_egs{}.fits".format(OUT_DIR, SUFFIX), overwrite=True)
id_names.remove('help_id')
master_catalogue.remove_columns(id_names)

## X - Adding HEALPix index

We are adding a column with a HEALPix index at order 13 associated with each source.

In [ ]:
master_catalogue.add_column(Column(
    data=coords_to_hpidx(master_catalogue['ra'], master_catalogue['dec'], order=13),
    name="hp_idx"
))

## XI - Saving the catalogue

In [ ]:
columns = ["help_id", "field", "ra", "dec", "hp_idx"]

bands = [column[5:] for column in master_catalogue.colnames if 'f_ap' in column]
for band in bands:
    columns += ["f_ap_{}".format(band), "ferr_ap_{}".format(band),
                "m_ap_{}".format(band), "merr_ap_{}".format(band),
                "f_{}".format(band), "ferr_{}".format(band),
                "m_{}".format(band), "merr_{}".format(band),
                "flag_{}".format(band)]    
    
columns += ["stellarity", "stellarity_origin", "flag_cleaned", "flag_merged", "flag_gaia", "flag_optnir_obs", 
            "flag_optnir_det", "zspec", "zspec_qual", "zspec_association_flag", "ebv"]

In [ ]:
# We check for columns in the master catalogue that we will not save to disk.
print("Missing columns: {}".format(set(master_catalogue.colnames) - set(columns)))

In [ ]:
master_catalogue[columns].write("{}/master_catalogue_egs{}.fits".format(OUT_DIR, SUFFIX))